In [58]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("asaniczka/mammals-image-classification-dataset-45-animals")

print("Path to dataset files:", path)

100%|██████████| 185M/185M [00:11<00:00, 17.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/asaniczka/mammals-image-classification-dataset-45-animals/versions/1


In [59]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

In [60]:
os.listdir(path)

['mammals']

In [61]:
os.listdir(os.path.join(path,'mammals'))

['jackal',
 'yak',
 'mongoose',
 'groundhog',
 'walrus',
 'camel',
 'porcupine',
 'giraffe',
 'vampire_bat',
 'red_panda',
 'armadillo',
 'vicuna',
 'zebra',
 'otter',
 'mountain_goat',
 'kangaroo',
 'anteater',
 'opossum',
 'tapir',
 'alpaca',
 'seal',
 'manatee',
 'warthog',
 'brown_bear',
 'dolphin',
 'water_buffalo',
 'horse',
 'american_bison',
 'arctic_fox',
 'sea_lion',
 'highland_cattle',
 'wombat',
 'sugar_glider',
 'wildebeest',
 'baboon',
 'squirrel',
 'african_elephant',
 'blue_whale',
 'koala',
 'badger',
 'weasel',
 'snow_leopard',
 'polar_bear',
 'rhinoceros',
 'orangutan']

In [62]:
generatort = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.2,
    rescale=1/255,
)
train = generatort.flow_from_directory(
    os.path.join(path,'mammals'),
    target_size=(224,224),
    batch_size=32,
    subset='training'
)
valid = generatort.flow_from_directory(
    os.path.join(path,'mammals'),
    target_size=(224,224),
    batch_size=32,
    subset='validation'
)


Found 11020 images belonging to 45 classes.
Found 2731 images belonging to 45 classes.


In [ ]:
test = valid.take(int(len(valid) * 0.5))
valid = valid.skip(int(len(valid) * 0.5))

In [64]:
class_name = test.class_names

In [ ]:
len(train), len(valid), len(test)

(21, 3, 3)

In [ ]:
#train_ds = train.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
#test_ds = test.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
#val_ds = valid.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
import tensorflow as tf

rescale_resize = tf.keras.Sequential([
    tf.keras.layers.Resizing(224, 224),
    tf.keras.layers.Rescaling(1.0/255)
])

In [86]:
pre_trained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet",
    pooling='max',
)

In [87]:
for layers in pre_trained_model.layers[:-2]:
    layers.trainable=False

In [88]:
model = tf.keras.Sequential([
    pre_trained_model,
    tf.keras.layers.Flatten(), # Flatten the output from the pre-trained model
    tf.keras.layers.Dense(45, activation='softmax') # Add a Dense layer for classification with 45 units and softmax activation
])


In [89]:
model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 1280)                │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 45)                  │          57,645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,315,629 (8.83 MB)

 Trainable params: 57,645 (225.18 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [90]:

from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(),
             loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy',patience=3)

In [91]:
histroy = model.fit(train,validation_data=valid,epochs=50,callbacks=[callback], batch_size=32)

Epoch 1/50
345/345 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5098 - loss: 2.9385

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


345/345 ━━━━━━━━━━━━━━━━━━━━ 39s 93ms/step - accuracy: 0.5103 - loss: 2.9346 - val_accuracy: 0.8118 - val_loss: 0.9140
Epoch 2/50
345/345 ━━━━━━━━━━━━━━━━━━━━ 21s 60ms/step - accuracy: 0.8771 - loss: 0.4928 - val_accuracy: 0.8440 - val_loss: 0.7245
Epoch 3/50
345/345 ━━━━━━━━━━━━━━━━━━━━ 24s 68ms/step - accuracy: 0.9137 - loss: 0.3231 - val_accuracy: 0.8418 - val_loss: 0.7882
Epoch 4/50
345/345 ━━━━━━━━━━━━━━━━━━━━ 22s 64ms/step - accuracy: 0.9424 - loss: 0.1943 - val_accuracy: 0.8649 - val_loss: 0.7297
Epoch 5/50
345/345 ━━━━━━━━━━━━━━━━━━━━ 21s 62ms/step - accuracy: 0.9675 - loss: 0.1056 - val_accuracy: 0.8568 - val_loss: 0.7661
Epoch 6/50
345/345 ━━━━━━━━━━━━━━━━━━━━ 21s 61ms/step - accuracy: 0.9645 - loss: 0.1178 - val_accuracy: 0.8704 - val_loss: 0.7396
Epoch 7/50
345/345 ━━━━━━━━━━━━━━━━━━━━ 43s 67ms/step - accuracy: 0.9706 - loss: 0.0935 - val_accuracy: 0.8685 - val_loss: 0.7601
Epoch 8/50
345/345 ━━━━━━━━━━━━━━━━━━━━ 22s 64ms/step - accuracy: 0.9785 - loss: 0.0680 - val_accurac